In [42]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import os

## Counties

In [43]:
# Load fips codes from census
state_fips = (
    pd.read_csv(
        "data/state_fips.txt", 
        delimiter="|",
        dtype="str",
    )
    .rename(
        columns = {
            "STATE": "STATEFP"
        }
    )
)

state_fips.head(1)

,STATEFP,STUSAB,STATE_NAME,STATENS
0,01,AL,Alabama,01779775


In [44]:
# load county geojson
counties = gpd.read_file("data/counties-geojson.json").drop(
    columns = ["AFFGEOID", "GEOID", "LSAD", "ALAND", "AWATER"]
)

# find center of plot and flip y axis

b = counties.total_bounds

center = (
    (b[0] + b[2]/2), 
    (b[1] + b[3]/2)
)

# Flip y-axis of geometry column
counties['geometry'] = counties.scale(
     yfact = -1,
     origin = center
)

In [46]:
# make a destination folder
try:
    os.mkdir("output/counties")
except FileExistsError:
    pass

In [47]:
# merge geodataframe with fips file and write out 
(
    counties.merge(
        state_fips,
        how = "left",
        on = "STATEFP"
    )
    .rename(
        columns = {
            "STATEFP": "state_fips",
            "COUNTYFP": "county_fips",
            "COUNTYNS": "county_gnis",
            "NAME": "county_name",
            "STUSAB": "state_abv",
            "STATE_NAME": "state_name",
            "STATENS": "state_gnis"
        }
     )
    .to_file(
        "output/counties/counties.shp"
    )
)

## States

In [48]:
# load states geojson
states = gpd.read_file("data/states-geojson.json").drop(
    columns = ["AFFGEOID", "GEOID", "LSAD", "ALAND", "AWATER"]
)

# find center of plot and flip y axis

b = states.total_bounds

center = (
    (b[0] + b[2]/2), 
    (b[1] + b[3]/2)
)

# Flip y-axis of geometry column
states['geometry'] = states.scale(
     yfact = -1,
     origin = center
)

In [50]:
# make a destination folder
try:
    os.mkdir("output/states")
except FileExistsError:
    pass

In [56]:
# merge geodataframe with fips file and write out 
(
    states.merge(
        state_fips,
        how = "left",
        on = "STATEFP"
    )
    .drop(
        columns = ["COUNTYFP", "COUNTYNS", "NAME"]
    )
    .rename(
        columns = {
            "STATEFP": "state_fips",
            "STUSAB": "state_abv",
            "STATE_NAME": "state_name",
            "STATENS": "state_gnis"
        }
    )
    .to_file(
        "output/states/states.shp"
    )
)